In [6]:
import zipfile
import os

# Extract the dogs-vs-cats.zip file from the data folder
zip_path = './data/dogs-vs-cats.zip'
extract_path = './data/'

if os.path.exists(zip_path):
    try:
        # Check if file is not empty
        if os.path.getsize(zip_path) == 0:
            print(f"Error: Zip file is empty (0 bytes). Please download the dogs-vs-cats dataset.")
        else:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
            print(f"Successfully extracted {zip_path}")
            print("Contents:", os.listdir(extract_path))
    except zipfile.BadZipFile:
        print(f"Error: {zip_path} is corrupted or not a valid zip file. Please download it again.")
    except Exception as e:
        print(f"Error: {e}")
else:
    print(f"Zip file not found at {zip_path}")

Error: Zip file is empty (0 bytes). Please download the dogs-vs-cats dataset.


In [3]:
# For local environment: specify your data directory path
# uploaded = './data/'  # or your desired data path
print("Using local environment - set data paths as needed")

Using local environment - set data paths as needed


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
import itertools
import copy
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [7]:
transform_cifar = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_cifar = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
test_cifar  = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)

train_loader_cifar = DataLoader(train_cifar, batch_size=64, shuffle=True)
test_loader_cifar  = DataLoader(test_cifar, batch_size=64, shuffle=False)


100%|██████████| 170M/170M [10:25<00:00, 273kB/s]  
/Users/aryan/DEEP-LEARNING LAB/.venv/lib/python3.13/site-packages/torchvision/datasets/cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import os
import shutil
import zipfile

transform_dogs = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

# Extract and organize dogs-vs-cats dataset
zip_dir = './data/dogs-vs-cats'
train_zip = os.path.join(zip_dir, 'train.zip')
test_zip = os.path.join(zip_dir, 'test1.zip')
train_path = os.path.join(zip_dir, 'train')
test_path = os.path.join(zip_dir, 'test')

# Extract train.zip if not already extracted
if os.path.exists(train_zip) and not os.path.exists(os.path.join(zip_dir, 'train', 'cats')):
    with zipfile.ZipFile(train_zip, 'r') as zip_ref:
        zip_ref.extractall(zip_dir)
    print("Extracted train.zip")

# Extract test1.zip if not already extracted
if os.path.exists(test_zip) and not os.path.exists(os.path.join(zip_dir, 'test_organized')):
    with zipfile.ZipFile(test_zip, 'r') as zip_ref:
        zip_ref.extractall(zip_dir)
    os.rename(os.path.join(zip_dir, 'test1'), os.path.join(zip_dir, 'test_organized'))
    print("Extracted and renamed test1.zip to test_organized")

# Organize train directory into class folders (cats/dogs) if needed
train_cats_dir = os.path.join(train_path, 'cats')
train_dogs_dir = os.path.join(train_path, 'dogs')

if not os.path.exists(train_cats_dir):
    os.makedirs(train_cats_dir, exist_ok=True)
if not os.path.exists(train_dogs_dir):
    os.makedirs(train_dogs_dir, exist_ok=True)
    
    # Move cat and dog images to respective folders
    for file in os.listdir(train_path):
        if file.startswith('cat.') and file.endswith('.jpg'):
            shutil.move(os.path.join(train_path, file), os.path.join(train_cats_dir, file))
        elif file.startswith('dog.') and file.endswith('.jpg'):
            shutil.move(os.path.join(train_path, file), os.path.join(train_dogs_dir, file))
    print("Organized train data into cats and dogs folders")

# Load training dataset using ImageFolder
train_dogs = datasets.ImageFolder(train_path, transform=transform_dogs)

# For test data (which has no labels), create dummy labels directory structure
test_dummy_path = os.path.join(zip_dir, 'test_dummy')
test_dummy_images = os.path.join(test_dummy_path, 'images')
if not os.path.exists(test_dummy_images):
    os.makedirs(test_dummy_images, exist_ok=True)
    # Copy all test images to a single class directory
    test_org_path = os.path.join(zip_dir, 'test_organized')
    for file in os.listdir(test_org_path):
        if file.endswith('.jpg'):
            src = os.path.join(test_org_path, file)
            dst = os.path.join(test_dummy_images, file)
            if not os.path.exists(dst):
                shutil.copy(src, dst)
    print("Organized test data")

test_dogs = datasets.ImageFolder(test_dummy_path, transform=transform_dogs)

train_loader_dogs = DataLoader(train_dogs, batch_size=32, shuffle=True)
test_loader_dogs  = DataLoader(test_dogs, batch_size=32, shuffle=False)

print("Classes:", train_dogs.classes)
print(f"Training samples: {len(train_dogs)}, Test samples: {len(test_dogs)}")

Extracted train.zip
Extracted and renamed test1.zip to test
Organized train data into cats and dogs folders


FileNotFoundError: Couldn't find any class folder in ./data/dogs-vs-cats/test.

In [ ]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes, activation):
        super(CustomCNN, self).__init__()

        if activation == "relu":
            act = nn.ReLU()
        elif activation == "tanh":
            act = nn.Tanh()
        else:
            act = nn.LeakyReLU()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            act,
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            act,
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            act,
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*8*8, 256),
            act,
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [ ]:
def init_weights(model, init_type):
    for m in model.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            if init_type == "xavier":
                nn.init.xavier_uniform_(m.weight)
            elif init_type == "kaiming":
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
            else:
                nn.init.normal_(m.weight, mean=0.0, std=0.02)


In [ ]:
def train_model(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0
    correct = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

    return running_loss/len(loader), correct/len(loader.dataset)


def eval_model(model, loader, criterion):
    model.eval()
    running_loss = 0
    correct = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

    return running_loss/len(loader), correct/len(loader.dataset)


In [ ]:
activations = ["relu", "tanh", "leaky"]
inits = ["xavier", "kaiming", "random"]
optimizers_list = ["sgd", "adam", "rmsprop"]

criterion = nn.CrossEntropyLoss()

best_models = {}

for act, init, opt in itertools.product(activations, inits, optimizers_list):
    print(f"\nConfig: {act} | {init} | {opt}")

    model = CustomCNN(num_classes=10, activation=act).to(device)
    init_weights(model, init)

    if opt == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=0.01)
    elif opt == "adam":
        optimizer = optim.Adam(model.parameters(), lr=0.001)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=0.001)

    for epoch in range(5):
        train_loss, train_acc = train_model(model, train_loader_cifar, criterion, optimizer)
        val_loss, val_acc = eval_model(model, test_loader_cifar, criterion)
        print(f"Epoch {epoch+1}: Val Acc={val_acc:.4f}")

    best_models[f"{act}_{init}_{opt}"] = copy.deepcopy(model.state_dict())


In [ ]:
  resnet = models.resnet18(pretrained=True)

  for param in resnet.parameters():
      param.requires_grad = False

  resnet.fc = nn.Linear(resnet.fc.in_features, 10)
  resnet = resnet.to(device)

  optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)

  for epoch in range(5):
      train_loss, train_acc = train_model(resnet, train_loader_cifar, criterion, optimizer)
      val_loss, val_acc = eval_model(resnet, test_loader_cifar, criterion)
      print(f"ResNet Epoch {epoch+1} Acc={val_acc:.4f}")
